In [64]:
%matplotlib inline
import sys,os,argparse
sys.path.append("../")
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import view
reload(view)

dd = pd.read_csv("../all.csv")
#print d.columns

In [65]:
d = view.DataAccessor()
d.loadCSV("../all.csv")
d.loadW2V("../w2v/wiki_mincount100.w2v")
t = view.view(d,columnPairs=(u"Acquiror business description(s)",u"Target business description(s)"),nBatch=256,nLength=20,reloadPath="../models/BusinessDescription_minCount100_nLength20_nGRU512/weights.2329.hdf5")

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 20, 100)       0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 20, 100)       0                                            
____________________________________________________________________________________________________
sequential_9 (Sequential)        (None, 64)            1948736     input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
distance (Lambda)                (None, 1)             0           sequential_9[1][0]      

In [45]:
score,text,index = t.testAllFromStr("Chemicals distributor, Rubber and latex products distributor")

In [46]:
df = pd.DataFrame({"score":score,"inex":index,"score":text})
df.to_csv("1.csv")
print df.sort(columns="score")

          inex                                              score
71630    71630  1-net solution consulting services, Homepage d...
12234    12234  10 KV box transformer substation manufacturer,...
12232    12232  10 KV box transformer substation manufacturer,...
12231    12231  10 KV box transformer substation manufacturer,...
12230    12230  10 KV box transformer substation manufacturer,...
12229    12229  10 KV box transformer substation manufacturer,...
12228    12228  10 KV box transformer substation manufacturer,...
12227    12227  10 KV box transformer substation manufacturer,...
12226    12226  10 KV box transformer substation manufacturer,...
12235    12235  10 KV box transformer substation manufacturer,...
12233    12233  10 KV box transformer substation manufacturer,...
76000    76000                              100 yen shop operator
76001    76001                              100 yen shop operator
60018    60018                             100 yen store operator
77114    7

/home/ysasaki/.local/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [50]:
print d.d[u"Acquiror business description(s)"][index]

0         Biopharmaceuticals developer, Biopharmaceutica...
1         Food condiments and sauces manufacturer holdin...
2         Food condiments and sauces manufacturer holdin...
3                         Beer manufacturer holding company
4                         Beer manufacturer holding company
5                         Beer manufacturer holding company
6                         Beer manufacturer holding company
7                         Beer manufacturer holding company
8                         Beer manufacturer holding company
9                         Beer manufacturer holding company
10                        Beer manufacturer holding company
11                        Beer manufacturer holding company
12                        Beer manufacturer holding company
13                        Beer manufacturer holding company
14                        Beer manufacturer holding company
15                        Beer manufacturer holding company
16                        Beer manufactu

In [93]:
print d.d[u"Acquiror business description(s)"][40873]
print d.d[u"Target business description(s)"][40873]

print
print d.d[u"Acquiror business description(s)"][1]
print d.d[u"Target business description(s)"][1]

print
print d.d[u"Acquiror business description(s)"][27]
print d.d[u"Target business description(s)"][27]

print
print d.d[u"Acquiror business description(s)"][28]
print d.d[u"Target business description(s)"][28]

print
print d.d[u"Acquiror business description(s)"][271]
print d.d[u"Target business description(s)"][271]
 
print
print d.d[u"Acquiror business description(s)"][272]
print d.d[u"Target business description(s)"][272]
 
print
print d.d[u"Acquiror business description(s)"][0]
print d.d[u"Target business description(s)"][0]

Investment holding company
Chemicals distributor, Rubber and latex products distributor

Food condiments and sauces manufacturer holding company, Soft and spreadable cheeses manufacturer holding company, Soft drinks manufacturer holding company
Personal care products and consumer goods manufacturer

Broadband telecommunications services holding company, Cable television services holding company, High-speed data and voice services holding company, Wireless backhaul services holding company
Broadband cable video, data and voice services, Cable television network services

Broadband telecommunications services holding company, Cable television services holding company, High-speed data and voice services holding company, Wireless backhaul services holding company
Broadband cable video, data and voice services, Cable television network services

Broadband telecommunications services holding company, Cable television services holding company, High-speed data and voice services holding compan

In [66]:
t.testOnes(u"Target business description(s)",u"Chemicals distributor, Rubber and latex products distributor")

ValueError: shapes (1,64) and (1,64) not aligned: 64 (dim 1) != 1 (dim 0)

In [136]:
def testOnes(t1,t2,nLength=20):
    v1 = t.dAcc.str_to_w2v(t1,fixedLength=nLength)
    v2 = t.dAcc.str_to_w2v(t2,fixedLength=nLength)
    y1 = t.outX1.predict({"input_1":np.expand_dims(v1,axis=0)})[0]
    y2 = t.outX1.predict({"input_1":np.expand_dims(v2,axis=0)})[0]
    #print v1,v2
    #print np.dot(v1,v2)
    res = np.dot(y1,y2)
    #v1,v2 = v1[0],v2[0]
    #v1,v2 = np.sum(v1,axis=0)/np.sum(np.linalg.norm(v1,axis=0)>0),np.sum(v2,axis=0)/np.sum(np.linalg.norm(v2,axis=0)>0)
    def myMean(v):
        s = []
        c = 0
        for x in v:
            if np.linalg.norm(x)>0.:
                s.append(x)
                c += 1
        s = np.array(s)
        s = np.sum(s,axis=0)
        s = s/c

        s /= np.linalg.norm(s)

        return s
    
    v1,v2 = myMean(v1),myMean(v2)
    mmm = np.dot(v1,v2)
    
        
    #mmm = np.dot(v1,v2)/np.dot(v1,v1)/np.dot(v2,v2)
    print
    print res, mmm,"   ",t1,"   ",t2
    return
testOnes("Investment holding company","Chemicals distributor, Rubber and latex products distributor")
testOnes("Food condiments and sauces manufacturer holding company, Soft and spreadable cheeses manufacturer holding company, Soft drinks manufacturer holding company","Personal care products and consumer goods manufacturer")
testOnes("Broadband telecommunications services holding company, Cable television services holding company, High-speed data and voice services holding company, Wireless backhaul services holding company","Broadband cable video, data and voice services, Cable television network services")
testOnes("Broadband telecommunications services holding company, Cable television services holding company, High-speed data and voice services holding company, Wireless backhaul services holding company","Cable telecommunication systems operator")
testOnes("Broadband telecommunications services holding company, Cable television services holding company, High-speed data and voice services holding company, Wireless backhaul services holding company","Cable telecommunication systems operator, Multichannel video programming distribution services, Multichannel video programming production services, News and sports television broadcasting services, Web hosting and managed security, digital TV, business data and fibre services")

testOnes("Broadband telecommunications services holding company","Broadband telecommunications services holding company")
testOnes("Broadband telecommunications holding company","Broadband telecommunications services company")


testOnes("Broadband telecommunications","telecommunications services")

testOnes("Biopharmaceuticals developer, Biopharmaceuticals manufacturer, Consumer healthcare products manufacturer, Infant food manufacturer, Pharmaceutical products manufacturer","Pharmaceutical solutions research and development services, Pharmaceuticals manufacturer")
testOnes("Biopharmaceuticals","telecommunications")
testOnes("Food","telecommunications")
testOnes("Food","Investment")
testOnes("Broadband","telecommunications")
testOnes("Broadband food","telecommunications")

testOnes("Chemicals distributor", "Rubber and latex products distributor")
testOnes("Food distributor", "Rubber and latex products distributor")
testOnes("telecommunication", "Rubber and latex products distributor")
testOnes("Chemicals", "Rubber and latex products distributor")
testOnes("Chemicals", "food")
testOnes("Chemicals", "telecommunication")
#    t.test("Biopharmaceuticals developer, Biopharmaceuticals manufacturer, Consumer healthcare products manufacturer, Infant food manufacturer, Pharmaceutical products manufacturer")
#    t.test("Food condiments and sauces manufacturer holding company, Soft and spreadable cheeses manufacturer holding company, Soft drinks manufacturer holding company")
#    t.test("Broadband telecommunications services holding company, Cable television services holding company, High-speed data and voice services holding company, Wireless backhaul services holding company")



0.281243 0.594786     Investment holding company     Chemicals distributor, Rubber and latex products distributor

-0.500957 0.761297     Food condiments and sauces manufacturer holding company, Soft and spreadable cheeses manufacturer holding company, Soft drinks manufacturer holding company     Personal care products and consumer goods manufacturer

-0.260273 0.855174     Broadband telecommunications services holding company, Cable television services holding company, High-speed data and voice services holding company, Wireless backhaul services holding company     Broadband cable video, data and voice services, Cable television network services

0.351546 0.769795     Broadband telecommunications services holding company, Cable television services holding company, High-speed data and voice services holding company, Wireless backhaul services holding company     Cable telecommunication systems operator

0.0322355 0.803436     Broadband telecommunications services holding company, Cab